# Importing the House Price data from the Melbourne City Database (API)
(VB)

In [ ]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
# base URL for API
base_url = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/house-prices-by-small-area-sale-year/records?limit=50&refine=sale_year%3A"

In [ ]:
# get all years
min_year = 2000 - 1
max_year = 2016
temp = []

for i in range(max_year, min_year, -1):
    response = requests.get(base_url + str(i)).json()

    year_results = response["results"]

    for j in range(len(year_results)):
        temp.append(year_results[j])
    
house_price_df = pd.DataFrame(temp)
house_price_df.head()

In [ ]:
# Get data from DF
house_price_df.count()

In [ ]:
# Drop NA data
house_price_df = house_price_df.dropna(how='any')
house_price_df.count()

In [ ]:
# Display clean DF
house_price_df.head()

In [ ]:
# Get unique small areas
print(house_price_df["small_area"].unique())

# Modifying house_price_df for 2 year ranges, to compare with Income Data

In [ ]:
# Modifying house_price_df for 2 year ranges, to compare with Income Data
starting_years = list(range(2003,2015 + 1,2))

# empty list to store new dictionaries
median_price_two_year_dict = {}
sum_transactions_two_year_dict = {}

# loop through year ranges
for year in starting_years:
    st_year = house_price_df[house_price_df["sale_year"] == str(year)]
    en_year = house_price_df[house_price_df["sale_year"] == str(year + 1)]

    # get intersection
    two_year = pd.merge(st_year, en_year, how ='inner', on =['small_area', 'type'])

    # get rid of "Docklands House/Townhouse" and "Southbank House/Townhouse"
    two_year = two_year.drop(two_year[(two_year['small_area'] == 'Docklands') & (two_year['type'] == 'House/Townhouse')].index)
    two_year = two_year.drop(two_year[(two_year['small_area'] == 'Southbank') & (two_year['type'] == 'House/Townhouse')].index)
    two_year = two_year.reset_index(drop=True)

    # add together transaction counts, and average median prices
    average_median_2y = (two_year["median_price_x"] + two_year["median_price_y"])/2
    two_year["average_median_2y"] = average_median_2y
    sum_transactions_2y = two_year["transaction_count_x"] + two_year["transaction_count_y"]
    two_year["sum_transactions_2y"] = sum_transactions_2y

    # split into types
    two_year_groupby = two_year.groupby(["type"])
    two_year_ave_median = two_year_groupby["average_median_2y"].mean()
    two_year_sum_transactions = two_year_groupby["sum_transactions_2y"].mean()

    # put into dictionaries
    median_price_two_year_dict[f"%s-%s" % (year, year + 1)] = two_year_ave_median
    sum_transactions_two_year_dict[f"%s-%s" % (year, year + 1)] = two_year_sum_transactions



In [ ]:
# median price DF
median_price_two_year_df = pd.DataFrame(median_price_two_year_dict)
median_price_two_year_df.head(17)

In [ ]:
# number of transactions DF
sum_transactions_two_year_df = pd.DataFrame(sum_transactions_two_year_dict)
sum_transactions_two_year_df.head(17)

# Importing the Household Income data from the ABS
(LM)

In [ ]:
# Specify the path to your Excel file
household_income_data = "Resources/5204055011do001.xlsx"  

In [ ]:
# Define the row indices to skip
rows_to_skip = list(range(0, 6)) + [7] + list(range(9, 26)) + list(range(27, 40)) + list(range(42, 127))  

In [ ]:
# Read ABS income data file and store into Pandas DataFrames
dfs = pd.read_excel(household_income_data, sheet_name=None, skiprows=rows_to_skip)  

In [ ]:
# Create an empty list to store DataFrames with tab names as a column
dfs_with_tab_name = [] 

In [ ]:
# Flag to skip the first and last tab
skip_first_tab = True
skip_last_tab = False

In [ ]:
# Iterate over each sheet in the dictionary
for sheet_name, df in dfs.items():
    if skip_first_tab:
        skip_first_tab = False
        continue  # Skip processing the first tab
        
    if skip_last_tab:
        break  # Exit the loop if the last tab is reached
        
    # Determine the column indices dynamically based on the actual number of columns
    num_cols = min(7, len(df.columns))
    
    # Ensure we don't exceed the number of columns in the DataFrame
    columns_to_read = list(range(num_cols))
    
    # Select only the specified columns
    df_selected = df.iloc[:, columns_to_read].copy()
    
    # Add a new column with the tab name to the selected DataFrame using loc
    df_selected.loc[:, 'Year/s'] = sheet_name
    
    # Append the modified DataFrame to the list
    dfs_with_tab_name.append(df_selected)    

In [ ]:
# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs_with_tab_name, ignore_index=True)  

In [ ]:
# Define a dictionary to map old values to new values
value_mapping = {'Table 1.1': '2003-2004', 'Table 1.2': '2005-2006', 'Table 1.3': '2007-2008', 'Table 1.4': '2009-2010', 'Table 1.5': '2011-2012', 'Table 1.6': '2013-2014', 'Table 1.7': '2015-2016', 'Table 1.8': '2017-2018', 'Table 1.9': '2019-2020', 'Table 1.10': '2020-2021', 'Table 1.11': '2021-2022', 'Explanatory Notes': '', 'tab_name': 'Year/s'}

In [ ]:
# Replace values in column 'tab_name' using the value_mapping dictionary
combined_df['Year/s'] = combined_df['Year/s'].replace(value_mapping)

In [ ]:
# Remove rows where column 'Year/s' has value '2020-2021' or '2021-2022'
values_to_remove = ['2020-2021', '2021-2022']
cleaned_df = combined_df[~combined_df['Year/s'].isin(values_to_remove)]

In [ ]:
# Convert values to numeric data type
cleaned_df.loc[:, 'Wages and salaries'] = cleaned_df['Wages and salaries'].astype(float)
cleaned_df.loc[:, 'Income from  unincorporated business (a)'] = cleaned_df['Income from  unincorporated business (a)'].astype(float)
cleaned_df.loc[:, 'Property income and superannuation'] = cleaned_df['Property income and superannuation'].astype(float)
cleaned_df.loc[:, 'Government pensions and allowances'] = cleaned_df['Government pensions and allowances'].astype(float)
cleaned_df.loc[:, 'Other'] = cleaned_df['Other'].astype(float)
cleaned_df.loc[:, 'All households (b)'] = cleaned_df['All households (b)'].astype(float)
cleaned_df.loc[:, 'Year/s'] = cleaned_df['Year/s'].astype(str)

In [ ]:
# Specify the file path for the CSV file
csv_file_path = 'ABS_income_data.csv'  

In [ ]:
# Save the DataFrame to a CSV file
cleaned_df.to_csv(csv_file_path, index=False) 

In [ ]:
print(f"DataFrame successfully saved to '{csv_file_path}'.")

In [ ]:
# Create a pivot table with 'Category' as rows, 'Year' as columns, and 'Value' as values
pivot_df = cleaned_df.pivot_table(columns='Year/s', values='All households (b)', aggfunc='sum')


In [ ]:
# rename the index as "Household income"
pivot_df = cleaned_df.pivot_table(index='Unnamed: 0', columns='Year/s', values='All households (b)', aggfunc='sum')
pivot_df = pivot_df.rename_axis('Household income')

In [ ]:
household_income_df = pivot_df
household_income_df.head()

In [ ]:
# Transpose the DataFrame to swap rows and columns
transposed_df = cleaned_df.T.reset_index()

In [ ]:
# Filter the transposed DataFrame to include the rows with income categories based on a specific column
income_comparison_df = transposed_df[transposed_df['index'].isin(['Gross Disposable Income', 'Total Gross Income', 'Total Income Payable'])]

In [ ]:
# Set the 'Year' column as the index
income_comparison_df.set_index('index', inplace=True)

In [ ]:
# Drop the 'index' column after setting it as the index
income_comparison_df.drop(columns='index', inplace=True)

In [ ]:
# Plot the line graph
income_comparison_df.T.plot(kind='line', marker='o', figsize=(12, 6))
plt.xlabel('Year')
plt.ylabel('Income')
plt.title('Comparison of Income Categories Over the Years')
plt.legend(title='Income Category')
plt.grid(True)
plt.show()

In [ ]:
# Transpose the DataFrame to swap rows and columns
transposed_df = cleaned_df.T.reset_index()

In [ ]:
# Filter the transposed DataFrame to include the rows with income categories based on a specific column
income_comparison_df = transposed_df[transposed_df['index'].isin(['Gross Disposable Income', 'Total Gross Income', 'Total Income Payable'])]

In [ ]:
# Set the 'index' column as the index for the DataFrame
income_comparison_df.set_index('index', inplace=True)

In [ ]:
# Drop the column by position after setting it as the index
income_comparison_df = income_comparison_df.iloc[:, 1:]

In [ ]:
# Plot the line graph
income_comparison_df.T.plot(kind='line', marker='o', figsize=(12, 6))
plt.xlabel('Year')
plt.ylabel('Income')
plt.title('Comparison of Income Categories Over the Years')
plt.legend(title='Income Category')
plt.grid(True)
plt.show()

# Importing the Population and Suburb data from the ABS
(ED)

In [ ]:
# Files to Load
population_data_to_load_2021 = ("../Project_1/2021_GCP_SAL_for_VIC_short-header/2021 Census GCP Suburbs and Localities for VIC/2021Census_G03_VIC_SAL.csv")
suburb_name_data_to_load_2021 = ("../Project_1/2021_GCP_SAL_for_VIC_short-header/Metadata/2021Census_geog_desc_1st_2nd_3rd_release.xlsx")

In [ ]:
# Read Population data into Pandas DataFrames
population_data_2021_df = pd.read_csv(population_data_to_load_2021)

# Read population and Suburb data into Pandas DataFrames and specifiy which worksheet number to be read in suburb file
sheet_name = 0
suburb_name_data_2021_df = pd.read_excel("../Project_1/2021_GCP_SAL_for_VIC_short-header/Metadata/2021Census_geog_desc_1st_2nd_3rd_release.xlsx", sheet_name = 5)
# Reference:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

In [ ]:
# Test only: Check data load for suburb data
suburb_name_data_2021_df.head()

In [ ]:
# Rename column in Population data for allow DataFrames to merge
population_data_2021_df= population_data_2021_df.rename(columns={"SAL_CODE_2021":"Census_Code_2021"})
population_data_2021_df.head()

In [ ]:
# Merge Population Data and Suburb Data DataFrames
population_and_suburb_merged_df = pd.merge(population_data_2021_df, suburb_name_data_2021_df, how = "left", on=["Census_Code_2021", "Census_Code_2021"])

In [ ]:
# Set Index to AGSS_Code_2021 and rename columns to meaningful headings
population_and_suburb_data_df = population_and_suburb_merged_df.set_index(["AGSS_Code_2021"])
population_and_suburb_data_df = population_and_suburb_data_df.rename(columns={"Total_Total":"Total_Population","Census_Name_2021":"Suburb_Name"})

In [ ]:
# Create final DataFrame with only the required columns
population_and_suburb_df = population_and_suburb_data_df[["Census_Code_2021","Suburb_Name","Total_Population","Area sqkm"]]

In [ ]:
# Test only: Check final DataFrame with required columns
population_and_suburb_df.head()